# **Hands-on 4: Exemplo Lena-Simple-EPC**

### **Objetivo:**
A proposta deste Hands-on é descrever e explicar o código de exemplo Lena-Simple-EPC.cc e por consequência explicar o que é EPC (*Evolved Packet Core*). Além disso, objetiva-se construir um script de simulação para ser utilizado em uma campanha de convergência de métricas.


### **Cenário:**

O Evolved Packet Core (EPC) é uma estrutura destinada a permitir o uso de Protocolos de Internet (IPs) em aparelhos 4G Long-Term Evolution (LTE) para fornecer comunicação via voz e dados como uma única rede convergente. Neste Hands on veremos como é implementada esta tecnologia em um exemplo do ns3.

## **Requisitos:**

*   Ter instalado o ns-3.34 ou superior; e
*   Ter realizado as leituras preliminares necessárias.


## **Parte 1: Como funciona o Exemplo Lena-Simple-EPC.cc?**

Este *script* realiza a comunicação de dois UEs e uma rede de internet através de dois eNBs conectados ao EPC. O arranjo final é semelhante à ilustração abaixo.

![1.png](FIGS/1.png)

### **Passo 1: Entendendo o código.**

Inicialmente são determinados os includes das bibliotecas a serem utilizadas no exemplo e a descrição do using namespace ns3. 

![2.png](FIGS/2.png)

Após isso, a *main* é iniciada e são definidas as variáveis que serão utilizadas em outras partes do script.

![3.png](FIGS/3.png)

As linhas de código abaixo estabelecem as variáveis que poderão ser alteradas via terminal.

![4.png](FIGS/4.png)

Caso o boleano **useCa** for configurado para “True” a seguinte função if é utilizada:

![5.png](FIGS/5.png)

Primeiramente, além do LteHelper, já apresentado no exemplo Lena-Simple, utilizaremos uma classe EpcHelper adicional, que se encarregará de criar as entidades EPC e a topologia da rede. Neste caso, não é possível utilizar o EpcHelper diretamente, pois é uma classe base abstrata, portanto, é necessário utilizar uma de suas classes filhas, que fornecem diferentes implementações de topologia EPC. 

Neste exemplo, foi considerado o PointToPointEpcHelper, que implementa um EPC baseado em links ponto a ponto.

Após isso, é “avisado” ao **LTE Helper** que o **EPC** será utilizado por meio da linha de código lteHelper->SetepcHelper(epcHelper). Essa etapa é necessária para que o **LTE Helper** acione a configuração **EPC** apropriada em correspondência a alguma configuração importante, como quando um novo eNB ou UE é adicionado ou um EPS bearer é criado. O **EPC Helper** cuidará automaticamente das configurações necessárias, como a criação do link S1 e a configuração do portador S1, sem que haja intervenção do usuário.

É importante salientar que, chamar a linha de código supracitada ativa o uso do **EPC** e tem o efeito colateral de que qualquer novo **LteEnbRrc** criado terá o atributo **EpsBearerToRlcMapping** definido como *RLC_UM_ALWAYS* em vez de RLC_SM_ALWAYS, se o último for o padrão; caso contrário, o atributo não será alterado (por exemplo, se você alterou o padrão para *RLC_AM_ALWAYS*, ele não será alterado).

![6.png](FIGS/6.png)

Deve-se notar que o **EpcHelper** também criará automaticamente o nó **PGW** e o configurará para que possa lidar adequadamente com o tráfego da rede **LTE**. Ainda assim, é necessário utilizar a linha de código abaixo para conectar o **PGW** a outras redes **IPv4/IPv6** (por exemplo, a internet, ou a outro **EPC**).

![7.png](FIGS/7.png)

Com isso, é criado um único RemoteHost.

![8.png](FIGS/8.png)

Assim, pode-se configurar todos os aspectos básicos da conexão e da internet, como o throughput, tamanho do arquivo e atraso do sistema. Depois é feita a conexão do **PGW** com a internet.

![9.png](FIGS/9.png)

Nas seguintes linhas de código, são criadas através do método **NodeContainer** e posicionadas através do método **MobilityModel** as **eNBs** e **UEs** da rede **LTE** estruturada. É interessante notar que os vetores são implementados por apenas uma variável na direção “x” que começa em 0 e é acrescida em 1 a cada iteração do laço **for**, enquanto as direções “y” e “z” são sempre zero.

![10.png](FIGS/10.png)

Em seguida, os *devices* LTE são instalados nos nós criados.

![11.png](FIGS/11.png)

As linhas de código abaixo configuram as UEs para rede IP. Em primeiro lugar, é chamado o método *AssignUeIpv4Adress* para gerenciar os endereços de Ips, após isso, são utilizados duas funções **for**, a primeira é utilizado para implementar os endereços de Ip nas UEs, e o segundo conectará as UEs a seus respectivos eNodeBs.

![12.png](FIGS/12.png)

Após isso, os aplicativos são instalados nos nós UE LTE.

![13.png](FIGS/13.png)

O exemplo ainda nos permite decidir o *status* do downlink, uplink e da comunicação entre os nodes entre ativado ou desativado, modificando as variáveis booleanas implementadas logo após o início da main.

![14.png](FIGS/14.png)

Finalmente, é configurado o tempo de funcionamento das aplicações *server* e *client*, definido o tempo final da simulação - por meio da variável estabelecida no começo do script - e dado o comando de *start* e *destroy* na simulação. 

![15.png](FIGS/15.png)

### **Passo 2: Executando o script.**

Inicialmente, copie o exemplo **lena-simple-epc.cc** para a pasta scratch. Após isso, crie uma pasta chamada **“resultados_lena-simple-epc”** no diretório principal do seu ns-3. Assim, execute a linha de comando abaixo.

```
./ns3 run "lena-simple-epc" --cwd="Resultado-lena-simple-epc”
```

![16.png](FIGS/16.png)

Assim, são gerados os mesmos tipos de arquivos gerados pelo exemplo **lena-simple.cc.**

![17.png](FIGS/17.png)

Por padrão, a execução do script resulta nos dados do intervalo de tempo entre 0.5 e 1.1 segundos da simulação e o espaçamento entre cada medida é de 0.1 segundos. Esses valores podem ser alterados substituindo os valores das variáveis **simTime** e **interPacketInterval** e alterando o tempo em que as aplicações irão iniciar.

### **Passo 3: Adicionando o REM.**

Copie o exemplo **lena-simple-epc** para a pasta scratch e renomeie-o **lena-simple-epc-rem**, após isso, introduza as linhas de código abaixo

```
Ptr<RadioEnvironmentMapHelper> remHelper = CreateObject<RadioEnvironmentMapHelper> ();
remHelper->SetAttribute ("Channel", PointerValue (lteHelper->GetDownlinkSpectrumChannel ()));
remHelper->SetAttribute ("OutputFile", StringValue ("rem.out"));
remHelper->SetAttribute ("XMin", DoubleValue (-400.0));
remHelper->SetAttribute ("XMax", DoubleValue (400.0));
remHelper->SetAttribute ("YMin", DoubleValue (-300.0));
remHelper->SetAttribute ("YMax", DoubleValue (300.0));
remHelper->SetAttribute ("Z", DoubleValue (0.0));
remHelper->Install ();

```
logo antes das linhas de código

```
Simulator::Stop (simTime);
Simulator::Run ();
```
Após isso, comente a seguinte linha de código 
```
lteHelper->EnableTraces ();
```
de forma que ela fique como mostrado abaixo.

![18.png](FIGS/18.png)

Fazemos isso pois a ativação do REM pode gerar traces incompletas e não precisaremos destes arquivos no momento. Após isso, crie uma nova pasta chamada **“resultados_lena-simple-epc-rem”** abra o terminal na pasta principal do ns-3 e execute a linha de comando

```
./ns3 run "lena-simple-epc-rem" --cwd="resultados_lena-simple-epc-rem”
```
Você perceberá que apenas o arquivo rem.out foi gerado na pasta.

![19.png](FIGS/19.png)

Após abrir o terminal na pasta onde está o arquivo rem.out, abra o gnuplot e, então, insira os comandos abaixo.


```
set view map;
set term x11;
set xlabel "X"
set ylabel "Y"
set cblabel "SINR (dB)"
plot "rem.out" using ($1):($2):(10*log10($4)) with image
```
A figura obtida deste passo será como a mostrada abaixo. 

![20.png](FIGS/20.png)

### **Passo 4: Extrair as informações apenas do downlink.**

A análise que se deseja realizar neste hands-on é sobre o *throughput* do *downlink* dos UEs estabelecidos. Para isso, serão analisados os dados presentes no arquivo DlTxPhyStats.Visto isso, não é necessário obter os arquivos contendo informação de *uplink* e comunicação entre os UEs. Para isso, apague os arquivos da pasta **“resultados_lena-simple-epc”** e altere as variáveis **disableUp** e **disablePl** pelo terminal quando for realizar a execução do script usando a linha de comando abaixo.

```
./ns3 run "lena-simple-epc --disableUl=true --disablePl=true" --cwd="Resultado-lena-simple-epc”
```
![21.png](FIGS/21.png)

## **Parte 2: Modificando o exemplo para uma campanha.**

### **Passo 1: Atualizar as variáveis do script.**

Para realizar uma campanha com base neste script é necessário que todas as variáveis que se deseja modificar para gerar diferentes cenários nas simulações estejam declaradas e com suas modificações habilitadas via linha de comando. 

Para isso, substitua os *includes* e as listas de variáveis do código original pelas linhas abaixo.




In [ ]:
#include "ns3/core-module.h"
#include "ns3/point-to-point-module.h"
#include "ns3/internet-module.h"
#include "ns3/applications-module.h"
#include "ns3/mobility-module.h"
#include "ns3/config-store-module.h"
#include "ns3/lte-module.h"
//#include "ns3/gtk-config-store.h"

#include "ns3/flow-monitor-module.h"
#include "ns3/flow-monitor-helper.h"
#include "ns3/flow-probe.h"

using namespace std;
using namespace ns3;

int
main (int argc, char *argv[])
{
	string fadingEnd = "/home/mcmuniz/ns-allinone-3.36/ns-3.36/src/lte/model/fading-traces";
/*string fadingEnd = "/home/marilia/ns-3.36/ns-allinone-3.36/ns-3.36/src/lte/model/fading-traces";*/
	string lambda = "0.25";
	uint32_t packetSize = 125;
	uint16_t numEnbs = 1;
	uint16_t numUes = 1;
	double appStartTime = 0.01;
	double simTime = 10.0;
	std::string ssimTime = to_string(simTime);
	std::string snumEnbs = to_string(numEnbs);
	std::string snumUes = to_string(numUes);
	Time interPacketInterval = MilliSeconds(100);
	double radius = 10.0;
	double distance = 20;
	bool useUdp = false;
	bool usePoisson = true;
	bool useShadowing = false;
	bool useFading = true;
	bool useDl = true;
	bool useUl = false;
	bool usePl = false;
	bool verbose = false;

	std::string outputDir = "./";
	std::string outputDir2 = "Results/";
	std::string NameFile;

	std::ifstream src;
	std::fstream dst;

	NameFile = "SimTime" + ssimTime + "ueNum" + snumUes + "enbNum" + snumEnbs;

	// Command line arguments
	CommandLine cmd;
	cmd.AddValue("lambda", "Lambda to be used in the model traffic", lambda);
	cmd.AddValue("packetSize", "Packet size (in bytes) to be used in the traffic with poisson distribution", packetSize);
	cmd.AddValue("numEnbs", "Number of eNodeBs", numEnbs);
	cmd.AddValue("numUes", "Number of UE", numUes);
	cmd.AddValue("simTime", "Total duration of the simulation", simTime);
	cmd.AddValue("distance", "Distance between eNBs [m]", distance);
	cmd.AddValue("interPacketInterval", "Inter packet interval",interPacketInterval);
	cmd.AddValue("radius", "Radius of the UE's distribution circle", radius);
	cmd.AddValue("distance", "Distance between eNodeBs", distance);
	cmd.AddValue("useUdp", "Use classic Udp traffic in the application",useUdp);
	cmd.AddValue("usePoisson", "Allow use of poisson distribution in the application", usePoisson);
	cmd.AddValue("useShadowing", "Disable the shadowing parameter of the propagation loss model", useShadowing);
	cmd.AddValue("useFading", "Disable the shadowing parameter of the propagation loss model", useFading);
	cmd.AddValue("useDl", "Disable downlink data flows", useDl);
	cmd.AddValue("useUl", "Disable uplink data flows", useUl);
	cmd.AddValue("usePl", "Disable data flows between peer UEs", usePl);
	cmd.AddValue("verbose", "Allow debug LOGs", verbose);
	cmd.AddValue("outputDir", "",outputDir);
	cmd.AddValue("outputDir2", "",outputDir2);
	cmd.Parse(argc, argv);

  ConfigStore inputConfig;
  inputConfig.ConfigureDefaults ();

  // parse again so you can override default values from the command line
  cmd.Parse(argc, argv);


Todas essas variáveis serão posteriormente utilizadas em outras partes do script. Note que algumas variáveis “já existentes” foram alteradas a fim de simplificar a sua utilização no cenário que está sendo construído, o que necessitará que se **modifique o nome das variáveis presentes em outros locais do código**, **o endereço da pasta do computador** e **altere algumas das lógicas** que utilizavam as mesmas.

### **Passo 2: Adicionar *Fading* e *Shadowing*.**

O *fading* é adicionado da mesma forma que foi visto anteriormente no hands-on do exemplo **lena-fading.cc**, utilizando *fading traces*. Então, caso o valor da variável **useFading** for verdadeiro, as linhas de código com as configurações do fading devem ser ativadas. Repare que agora existe uma variável chamada **fadingEnd** que contém o endereço dos *fading traces* que devem ser utilizados na simulação.

Já o *shadowing* é um atributo do modelo de perda de propagação ns3::ThreeGppUmaPropagationLossModel, desta forma, é necessário, antes de tudo, adicionar esse modelo ao script. Após isso, como o shadowing é **nativamente ativo**, caso o valor da variável **useShadowing** for falso, é necessário configurar o atributo **"ShadowingEnabled"** como falso. A imagem abaixo apresenta as linhas de código que precisam ser adicionadas e onde as mesmas devem ser situadas.

![22.png](FIGS/22.png)

In [ ]:
Ptr<LteHelper> lteHelper = CreateObject<LteHelper> ();

  if(useFading == true){
	  lteHelper->SetAttribute ("FadingModel", StringValue ("ns3::TraceFadingLossModel"));
	  lteHelper->SetFadingModelAttribute ("TraceFilename", StringValue (fadingEnd));
	  // these parameters have to be set only in case of the trace format
	  // differs from the standard one, that is
	  // - 10 seconds length trace
	  // - 10,000 samples
	  // - 0.5 seconds for window size
	  // - 100 RB
	  lteHelper->SetFadingModelAttribute ("TraceLength", TimeValue (Seconds (10.0)));
	  lteHelper->SetFadingModelAttribute ("SamplesNum", UintegerValue (10000));
	  lteHelper->SetFadingModelAttribute ("WindowSize", TimeValue (Seconds (0.5)));
	  lteHelper->SetFadingModelAttribute ("RbNum", UintegerValue (100));

  }

  lteHelper->SetAttribute("PathlossModel",StringValue ("ns3::ThreeGppUmaPropagationLossModel"));

  if(useShadowing == false){
	  lteHelper->SetPathlossModelAttribute("ShadowingEnabled",BooleanValue (false));
  }

  Ptr<PointToPointEpcHelper> epcHelper = CreateObject<PointToPointEpcHelper> ();
  lteHelper->SetEpcHelper (epcHelper);

### **Passo 3: Adicionar diferentes EPS Bearers.**

Quando utilizamos EPC a ativação de bearers é feita de uma maneira ligeiramente diferente em relação ao que foi feito para uma simulação somente LTE. Primeiro, o método ActivateDataRadioBearer não deve ser usado quando o EPC é usado. Além disso, quando o EPC é usado, o portador EPS padrão será ativado automaticamente quando você chamar LteHelper::Attach (). Todavia, se você deseja configurar um EPS bearer dedicado, pode fazê-lo usando o método LteHelper::ActivateDedicatedEpsBearer (). Este método toma como parâmetro o Traffic Flow Template (TFT), que é uma estrutura que identifica o tipo de tráfego que será mapeado para o EPS bearer dedicado. Desta forma, adicione as linhas de código abaixo logo antes das linhas de código que definem a aplicação.



In [ ]:
 // Attach one UE per eNodeB
  for (uint16_t i = 0; i < numEnbs; i++){
  	  for(uint16_t j = 0; j < numUes; j++){
  		  lteHelper->Attach(ueLteDevs.Get(j),enbLteDevs.Get(i));
  	  }
  }

  Ptr<EpcTft> tft = Create<EpcTft> ();
  EpcTft::PacketFilter pf;
  pf.localPortStart = 1100;
  pf.localPortEnd = 1100;
  tft->Add (pf);
  lteHelper->ActivateDedicatedEpsBearer (ueLteDevs,
                                         EpsBearer (EpsBearer::NGBR_VIDEO_TCP_DEFAULT),
                                         tft);


![23.png](FIGS/23.png)

### **Passo 4: Adicionar tráfego de Poisson**

Buscando utilizar um tráfego com um comportamento mais aproximado do que acontece na prática, pensou-se em adotar um tráfego que utiliza uma distribuição de Poisson. Para isso, utiliza-se um helper de aplicação que suporte o uso de variáveis aleatórias para reger o intervalo de criação dos pacotes. Assim, utiliza-se o OnOffHelper, configurando-o de modo que o tempo em Off é exponencial e o tempo On é longo o bastante para enviar um pacote. 

As linhas de código que definiam a aplicação no script original devem ser modificadas pelas linhas de código abaixo.

Repare que existe uma lógica envolvendo as variáveis **useUdp** e **usePoisson** para que apenas um dos tipos de tráfego possa ser utilizado por vez.

Além disso, algumas variáveis foram modificadas e outras adicionadas - como as variáveis **lambda, appStartTime, simTime** - para que se pudesse ter maior controle da simulação. 




In [ ]:
// Install and start applications on UEs and remote host
      uint16_t dlPort = 1100;
      uint16_t ulPort = 2000;
      uint16_t otherPort = 3000;
      ApplicationContainer clientApps;
      ApplicationContainer serverApps;
      if ((useUdp == true) && (usePoisson == false)) {
         	for (uint32_t u = 0; u < ueNodes.GetN(); ++u) {
         		if (useDl) {
         			PacketSinkHelper dlPacketSinkHelper("ns3::UdpSocketFactory", InetSocketAddress(Ipv4Address::GetAny(), dlPort));
         			serverApps.Add(dlPacketSinkHelper.Install(ueNodes.Get(u)));

         			UdpClientHelper dlClient(ueIpIface.GetAddress(u), dlPort);
         			dlClient.SetAttribute("Interval", TimeValue(interPacketInterval));
         			dlClient.SetAttribute("MaxPackets", UintegerValue(1000000));
         			clientApps.Add(dlClient.Install(remoteHost));
         		}

         		if (useUl) {
         			++ulPort;
         			PacketSinkHelper ulPacketSinkHelper("ns3::UdpSocketFactory", InetSocketAddress(Ipv4Address::GetAny(), ulPort));
         			serverApps.Add(ulPacketSinkHelper.Install(remoteHost));

         			UdpClientHelper ulClient(remoteHostAddr, ulPort);
         			ulClient.SetAttribute("Interval", TimeValue(interPacketInterval));
         			ulClient.SetAttribute("MaxPackets", UintegerValue(1000000));
         			clientApps.Add(ulClient.Install(ueNodes.Get(u)));
         		}

         		if (usePl && numUes > 1) {
         			++otherPort;
         			PacketSinkHelper packetSinkHelper("ns3::UdpSocketFactory", InetSocketAddress(Ipv4Address::GetAny(), otherPort));
         			serverApps.Add(packetSinkHelper.Install(ueNodes.Get(u)));

         			UdpClientHelper client(ueIpIface.GetAddress(u), otherPort);
         			client.SetAttribute("Interval", TimeValue(interPacketInterval));
         			client.SetAttribute("MaxPackets", UintegerValue(1000000));
         			clientApps.Add(client.Install(ueNodes.Get((u + 1) % numUes)));
         		}
         	}
         	serverApps.Start(MilliSeconds(appStartTime));
         	clientApps.Start(MilliSeconds(appStartTime));
         	serverApps.Stop(Seconds(simTime));
         	clientApps.Stop(Seconds(simTime));
         	lteHelper->EnableTraces();
         }

         if ((useUdp == false) && (usePoisson == true)) {
         	for (uint32_t u = 0; u < ueNodes.GetN(); ++u) {
         		if (useDl) {
         			PacketSinkHelper dlPacketSinkHelper("ns3::UdpSocketFactory", InetSocketAddress(ueIpIface.GetAddress(u), dlPort));
         			serverApps.Add(dlPacketSinkHelper.Install(ueNodes.Get(u)));

         			OnOffHelper dlClient("ns3::UdpSocketFactory", InetSocketAddress(ueIpIface.GetAddress(u), dlPort));
         			dlClient.SetAttribute("OnTime", StringValue("ns3::ConstantRandomVariable[Constant=0.001]"));
         			dlClient.SetAttribute("OffTime", StringValue("ns3::ExponentialRandomVariable[Mean=" + lambda + "]"));
         			dlClient.SetAttribute("PacketSize", UintegerValue(packetSize));
         			dlClient.SetAttribute("StartTime", TimeValue(MilliSeconds(100)));
         			dlClient.SetAttribute("StopTime", TimeValue(Seconds(simTime)));
         			clientApps.Add(dlClient.Install(remoteHost));
         		}
         		if (useUl) {
         			++ulPort;
         			PacketSinkHelper ulPacketSinkHelper("ns3::UdpSocketFactory", InetSocketAddress(remoteHostAddr, ulPort));
         			serverApps.Add(ulPacketSinkHelper.Install(remoteHost));

         			OnOffHelper ulClient("ns3::UdpSocketFactory", InetSocketAddress(remoteHostAddr, ulPort));
         			ulClient.SetAttribute("OnTime", StringValue("ns3::ConstantRandomVariable[Constant=0.001]"));
         			ulClient.SetAttribute("OffTime", StringValue("ns3::ExponentialRandomVariable[Mean=" + lambda + "]"));
         			ulClient.SetAttribute("PacketSize", UintegerValue(packetSize));
         			ulClient.SetAttribute("StartTime", TimeValue(MilliSeconds(100)));
         			ulClient.SetAttribute("StopTime", TimeValue(Seconds(simTime)));
         			clientApps.Add(ulClient.Install(ueNodes.Get(u)));
         		}

         	}
         	serverApps.Start(MilliSeconds(appStartTime));
         	clientApps.Start(MilliSeconds(appStartTime));
         	serverApps.Stop(Seconds(simTime));
         	clientApps.Stop(Seconds(simTime));
         	lteHelper->EnableTraces();
         	std::ostringstream oss;
         	    	oss << "/NodeList/"
         	    		<< remoteHost->GetId()
         	    		<< "/ApplicationList/0"
         	    		<< "/$ns3::OnOffApplication/Tx";
         	Config::Connect(oss.str(), MakeCallback(&TraceSentPacket));
         }

### **Passo 5: Adicionar TraceSentPacker.**

Para saber o exato momento em que o dado foi transmitido pela aplicação, sem os atrasos causados por outras camadas e, em outro momento, ter a possibilidade de averiguar se essa transmissão está realmente seguindo uma distribuição de Poisson, estabelecemos um *callback* que conecta o *trace sink* do Tx do OnOffHelper com a função TranceSentPacket, que gera um arquivo de mesmo nome contendo cada dado de tempo em que um dado foi enviado pela aplicação. Sabendo disso, insira as linhas de código abaixo **após os *includes*** e **acima da *main*** do *script*.



In [ ]:
NS_LOG_COMPONENT_DEFINE("LenaSimpleEpc");
std::ofstream m_TxTraceFile;
std::string m_TxTraceFileName;

void TraceSentPacket(std::string context, Ptr<const Packet > m_txTrace) {
//Vector position = model->GetPosition ();
//NS_LOG_UNCOND (context << "Time Tx: "
//              << Simulator::Now ().GetSeconds ()
//              << " Packet = " << m_txTrace);
    if (!m_TxTraceFile.is_open()) {
    	m_TxTraceFileName = "TxSentTrace.txt";
    	m_TxTraceFile.open(m_TxTraceFileName.c_str());
    	m_TxTraceFile << "Time" << std::endl;

    	if (!m_TxTraceFile.is_open()) {
    		NS_FATAL_ERROR("Could not open tracefile");
    	}
    }

    m_TxTraceFile << Simulator::Now() << std::endl;
}
// To be used in tic toc time counter
clock_t startTimer;
time_t beginTimer;

// Implementation of tic, i.e., start time counter
void tic() {
    beginTimer = time(&beginTimer);
    struct tm *timeinfo;
    timeinfo = localtime(&beginTimer);
    std::cout << "simulation start at: " << asctime(timeinfo) << std::endl;
}
// implementation of toc, i.e., stop time counter
    double toc() {
    time_t finishTimer = time(&finishTimer);
    double simTime = difftime(finishTimer, beginTimer) / 60.0;
    struct tm *timeinfo;
    timeinfo = localtime(&finishTimer);
    std::cout << "simulation finished at: " << asctime(timeinfo) << std::endl;
    //
    std::cout << "Time elapsed: " << simTime << " minutes" << std::endl;
    //
    return simTime;
}

Após isso, insira as seguintes linhas de código após as definições de start e stop time da aplicação com tráfego de Poisson, como mostra a figura abaixo.

In [ ]:
std::ostringstream oss;
    oss << "/NodeList/"
        << remoteHost->GetId()
        << "/ApplicationList/0"
        << "/$ns3::OnOffApplication/Tx";
Config::Connect(oss.str(), MakeCallback(&TraceSentPacket));

![24.png](FIGS/24.png)

### **Passo 6: Adicionar Flow Monitor**

Depois de termos obtido as métricas do traces fornecidos pelo próprio **LteHelper** e pelo *callback* que construímos, podemos obter ainda mais parâmetros do cenário construído utilizando o Flow Monitor, que é um *helper* que é configurado para este fim. Para adicioná-lo ao script, começa-se inserindo as linhas de código abaixo logo antes do **Simulator::Stop()**, como mostra a figura abaixo.



In [ ]:
FlowMonitorHelper flowmonHelper;
NodeContainer endpointNodes;
endpointNodes.Add(remoteHost);
endpointNodes.Add(ueNodes);

Ptr<ns3::FlowMonitor> monitor = flowmonHelper.Install(endpointNodes);
monitor->SetAttribute("DelayBinWidth", DoubleValue(0.001));
monitor->SetAttribute("JitterBinWidth", DoubleValue(0.001));
monitor->SetAttribute("PacketSizeBinWidth", DoubleValue(20));
AsciiTraceHelper asciiTraceHelper;

![25.png](FIGS/25.png)

Após isso, é necessário configurar o Flow Monitor. Para isso, insira as linhas de código abaixo após o **Simulator::Run()**.

In [ ]:
/*
      * To check what was installed in the memory, i.e., BWPs of eNb Device, and its configuration.
      * Example is: Node 1 -> Device 0 -> BandwidthPartMap -> {0,1} BWPs -> NrGnbPhy -> NrPhyMacCommong-> Numerology, Bandwidth, ...
       GtkConfigStore config;
       config.ConfigureAttributes ();
       */

      // Print per-flow statistics
      monitor->CheckForLostPackets();
       Ptr <Ipv4FlowClassifier> classifier = DynamicCast<Ipv4FlowClassifier>(flowmonHelper.GetClassifier());
      FlowMonitor::FlowStatsContainer stats = monitor->GetFlowStats();

      double averageFlowThroughput = 0.0;
      double averageFlowDelay = 0.0;

      /*
       std::ofstream outFile;
       std::string filename = outputDir + "/" + simTag;
       outFile.open (filename.c_str (), std::ofstream::out | std::ofstream::trunc);
       if (!outFile.is_open ())
       {
       std::cerr << "Can't open file " << filename << std::endl;
       return 1;
       }

       outFile.setf (std::ios_base::fixed);
       */

      for (std::map<FlowId,FlowMonitor::FlowStats>::const_iterator i = stats.begin(); i != stats.end(); ++i) {
      	Ipv4FlowClassifier::FiveTuple t = classifier->FindFlow(i->first);
      	std::stringstream protoStream;
      	protoStream << (uint16_t) t.protocol;
      	if (t.protocol == 6) {
      		protoStream.str("TCP");
      	}
      	if (t.protocol == 17) {
      		protoStream.str("UDP");
      	}
      	/*outFile << "Flow " << i->first << " (" << t.sourceAddress << ":" << t.sourcePort << " -> " << t.destinationAddress << ":" << t.destinationPort << ") proto " << protoStream.str () << "\n";
      	 outFile << "  Tx Packets: " << i->second.txPackets << "\n";
      	 outFile << "  Tx Bytes:   " << i->second.txBytes << "\n";
      	 outFile << "  TxOffered:  " << i->second.txBytes * 8.0 / (simTime - udpAppStartTime) / 1000 / 1000  << " Mbps\n";
      	 outFile << "  Rx Bytes:   " << i->second.rxBytes << "\n";*/
      	std::cout << "Flow " << i->first << " (" << t.sourceAddress << ":" << t.sourcePort << " -> " << t.destinationAddress << ":" << t.destinationPort << ") proto " << protoStream.str() << "\n";
      	std::cout << "  Tx Packets: " << i->second.txPackets << "\n";
      	std::cout << "  Tx Bytes:   " << i->second.txBytes << "\n";
      	std::cout << "  TxOffered:  " << i->second.txBytes * 8.0 / (simTime - appStartTime) / 1000 / 1000 << " Mbps\n";
      	std::cout << "  Rx Bytes:   " << i->second.rxBytes << std::endl;

      	if (i->second.rxPackets > 0) {
      		// Measure the duration of the flow from receiver's perspective
      		//double rxDuration = i->second.timeLastRxPacket.GetSeconds () - i->second.timeFirstTxPacket.GetSeconds ();
      		double rxDuration = (simTime - appStartTime);

      		averageFlowThroughput += i->second.rxBytes * 8.0 / rxDuration / 1000 / 1000;
      		averageFlowDelay += 1000 * i->second.delaySum.GetSeconds() / i->second.rxPackets;

      		/*outFile << "  Throughput: " << i->second.rxBytes * 8.0 / rxDuration / 1000 / 1000  << " Mbps\n";
      		 outFile << "  Mean delay:  " << 1000 * i->second.delaySum.GetSeconds () / i->second.rxPackets << " ms\n";
      		 //outFile << "  Mean upt:  " << i->second.uptSum / i->second.rxPackets / 1000/1000 << " Mbps \n";
      		 outFile << "  Mean jitter:  " << 1000 * i->second.jitterSum.GetSeconds () / i->second.rxPackets  << " ms\n";*/
      		std::cout << "  Throughput:  " << i->second.rxBytes * 8.0 / rxDuration / 1000 / 1000 << " Mbps\n";
      		std::cout << "  Mean delay:  " << 1000 * i->second.delaySum.GetSeconds() / i->second.rxPackets << " ms\n";
      		//std::cout << "  Mean upt:  " << i->second.uptSum / i->second.rxPackets / 1000/1000 << " Mbps \n";
      		std::cout << "  Mean jitter:  " << 1000 * i->second.jitterSum.GetSeconds() / i->second.rxPackets << " ms\n";

      	} else {
      		/*outFile << "  Throughput:  0 Mbps\n";
      		 outFile << "  Mean delay:  0 ms\n";
      		 outFile << "  Mean jitter: 0 ms\n";*/
      		std::cout << "  Throughput:  0 Mbps\n";
      		std::cout << "  Mean delay:  0 ms\n";
      		std::cout << "  Mean jitter: 0 ms\n";
      	}
      	//outFile << "  Rx Packets: " << i->second.rxPackets << "\n";
      	std::cout << "  Rx Packets: " << i->second.rxPackets << std::endl;
      }

      /*outFile << "\n\n  Mean flow throughput: " << averageFlowThroughput / stats.size () << "\n";
       outFile << "  Mean flow delay: " << averageFlowDelay / stats.size () << "\n";*/

      std::cout << "\n  Mean flow throughput: " << averageFlowThroughput / stats.size() << "\n";
      std::cout << "  Mean flow delay: " << averageFlowDelay / stats.size() << "\n";

      std::string dl_results, ul_results, dl_results2, ul_results2;
      dl_results = outputDir + "/" + "DL_" + NameFile + ".txt";
      ul_results = outputDir + "/" + "UL_" + NameFile + ".txt";
      //dl_results2 = outputDir2 + "/" + "DL_" + NameFile + ".txt";
      //ul_results2 = outputDir2 + "/" + "UL_" + NameFile + ".txt";

      Ptr<OutputStreamWrapper> DLstreamMetricsInit = asciiTraceHelper.CreateFileStream((dl_results));
      *DLstreamMetricsInit->GetStream()
      		<< "Flow_ID, Lost_Packets, Tx_Packets, Tx_Bytes, TxOffered(Mbps),  Rx_Packets, Rx_Bytes, T_put(Mbps), Mean_Delay_Rx_Packets, Mean_Jitter, Packet_Loss_Ratio"
      		<< std::endl;

      Ptr<OutputStreamWrapper>  ULstreamMetricsInit = asciiTraceHelper.CreateFileStream((ul_results));
      *ULstreamMetricsInit->GetStream()
      		<< "Flow_ID, Lost_Packets, Tx_Packets, Tx_Bytes, TxOffered(Mbps),  Rx_Packets, Rx_Bytes, T_put(Mbps), Mean_Delay_Rx_Packets, Mean_Jitter, Packet_Loss_Ratio"
      		<< std::endl;

      double statDurationTX = 0;
      double statDurationRX = 0;
      //Ptr classifier = DynamicCast(flowHelper.GetClassifier());
      //std::map stats = flowMonitor->GetFlowStats();
      uint16_t DlPort = 1234;
      uint16_t UlPort = DlPort + numEnbs * numUes + 1;
      for (std::map<FlowId,FlowMonitor::FlowStats>::const_iterator iter = stats.begin(); iter != stats.end(); ++iter) {
      	// some metrics calculation
      	statDurationRX = iter->second.timeLastRxPacket.GetSeconds() - iter->second.timeFirstTxPacket.GetSeconds();
      	statDurationTX = iter->second.timeLastTxPacket.GetSeconds()- iter->second.timeFirstTxPacket.GetSeconds();

      	double meanDelay, meanJitter, packetLossRatio, txTput, rxTput; //,NavComsumption,NavModemComsumption;
      	if (iter->second.rxPackets > 0) {
      		meanDelay = (iter->second.delaySum.GetSeconds() / iter->second.rxPackets);
      	} else // this value is set to zero because the STA is not receiving any packet
      	{
      		meanDelay = 0;
      	}
      	//
      	if (iter->second.rxPackets > 1) {
      		meanJitter = (iter->second.jitterSum.GetSeconds() / (iter->second.rxPackets - 1));
      	} else // this value is set to zero because the STA is not receiving any packet
      	{
      		meanJitter = 0;
      	}
      	//
      	if (statDurationTX > 0) {
      		txTput = iter->second.txBytes * 8.0 / statDurationTX / 1000 / 1000;
      	} else {
      		txTput = 0;
      	}
      	//
      	if (statDurationRX > 0) {
      		rxTput = iter->second.rxBytes * 8.0 / statDurationRX / 1000 / 1000;
      	} else {
      		rxTput = 0;
      	}
      	//
      	if ((iter->second.lostPackets > 0) & (iter->second.rxPackets > 0)) {
      		packetLossRatio = (double) (iter->second.lostPackets / (double) (iter->second.rxPackets + iter->second.lostPackets));
      	} else {
      		packetLossRatio = 0;
      	}
      	/*if(iter->first == auv.Get(0)->GetId()){
      	 NavComsumption = energyModel->GetTotalEnergyConsumption ();
      	 NavModemComsumption = basicSourcePtr ->GetInitialEnergy() - basicSourcePtr -> GetRemainingEnergy();
      	 }else{
      	 NavComsumption=0;
      	 NavModemComsumption=0;
      	 }*/
      	//
      	Ipv4FlowClassifier::FiveTuple t = classifier->FindFlow(iter->first);
      	//
      	Ptr <OutputStreamWrapper>  streamMetricsInit = NULL;
      	// Get file pointer for DL, if DL flow (using port and IP address to assure correct result)
      	std::cout << "\nFlow: " << iter->first << std::endl;
      	std::cout << "  t destination port " << t.destinationPort << std::endl;
      	std::cout << "  source address " << internetIpIfaces.GetAddress(1) << std::endl;
      	std::cout << "  t source address " << t.sourceAddress << std::endl;
      	std::cout << "  t destination port " << t.destinationPort << std::endl;
      	std::cout << "  sink address " << ueIpIface.GetAddress(0) << std::endl;
      	std::cout << "  t destination address " << t.destinationAddress << "\n";
      	if ((t.destinationPort == DlPort) || (t.sourceAddress == remoteHostAddr)) {
      		streamMetricsInit = DLstreamMetricsInit;
      		DlPort++;
      	}
      	// Get file pointer for UL, if UL flow (using port and IP address to assure correct result))
      	//else if ((t.destinationPort == UlPort)
      	else if ((t.destinationPort == UlPort) || (t.destinationAddress == remoteHostAddr)) {
      		streamMetricsInit = ULstreamMetricsInit;
      		UlPort++;
      	}
      	//
      	if (streamMetricsInit) {

      		*streamMetricsInit->GetStream() << (iter->first) << ", "
      				<< (iter->second.lostPackets) << ", "
      				//
      				<< (iter->second.txPackets) << ", "
      				//
      				<< (iter->second.txBytes) << ", "
      				//
      				<< txTput << ", "
      				//
      				<< (iter->second.rxPackets) << ", "
      				//
      				<< (iter->second.rxBytes) << ", "
      				//
      				<< rxTput << ", "
      				//
      				<< meanDelay << ", "
      				//
      				<< meanJitter << ", "
      				//
      				<< packetLossRatio
      				//
      				//<< NavComsumption << ", "
      				//
      				//<< NavModemComsumption
      				//
      				<< std::endl;
      	} else {
      		//TODO: chance for an ASSERT
      		if (true) {
      			std::cout << "Some problem to save metrics" << std::endl;
      			std::cout << "Flow ID: " << iter->first << ", Source Port: " << t.sourcePort << ", Destination Port: "
      					  << t.destinationPort << " (" << t.sourceAddress
      					  << " -> " << t.destinationAddress << ")" << std::endl;
      			std::cout << "gNB Address: " << t.destinationAddress << std::endl;
      			std::cout << "DLport: " << t.sourcePort << std::endl;
      			std::cout << "ULport: " << t.destinationPort << std::endl;
      		}
      	}

      	//m_bytesTotal =+ iter->second.rxPackets;
      }

      src.open(dl_results, std::ios::in | std::ios::binary);
      dst.open(dl_results2, std::ios::out | std::ios::binary);
      dst << src.rdbuf();

      src.close();
      dst.close();

      src.open(ul_results, std::ios::in | std::ios::binary);
      dst.open(ul_results2, std::ios::out | std::ios::binary);
      dst << src.rdbuf();

      src.close();
      dst.close();

      /*outFile.close ();

       std::ifstream f (filename.c_str ());

       if (f.is_open ())
       {
       std::cout << f.rdbuf ();
       }
       */
      toc();

  /*GtkConfigStore config;
  config.ConfigureAttributes();*/


O resultado será a criação de dois arquivos, um para *downlink* e outro para *uplink* cujos nomes serão determinados pelo tempo de simulação, número de UEs e número de eNBs.

Quando forem feitas todas as modificações no script lena simple epc, a versão final deve estar [dessa maneira](https://github.com/MariliaCostaMuniz/ic_LTE/blob/main/fase_01/HD_04/Campanha%20lena%20simple%20EPC.ipynb).

### **Passo 7: Executar o script novamente.**

Após ter concluído todos os passos anteriores, execute mais uma vez o *script* utilizando a linha de comando abaixo.

```
./ns3 run "my-lena-simple-epc --useUl=true --usePl=true" --cwd="Resultado-lena-simple-epc"
./waf --cwd="resultados_lena-simple-epc"/ --run "lena-simple-epc --disableUl=true --disablePl=true"
```
Os arquivos gerados devem ser os mostrados na figura abaixo.

![26.png](FIGS/26.png)

Repare que, por se tratar de uma simulação envolvendo apenas o *downlink*, o único arquivo gerado pelo **Flow Monitor** contendo informações será o desse enlace. A formatação desse arquivo é mostrada na figura abaixo.

![27.png](FIGS/27.png)

## **Parte 3: Criação da campanha.**
### **O que é uma campanha?**

Para que seja possível analisar um sistema é necessário reproduzir os resultados e investigar os comportamentos. Entretanto, a maioria das questões da natureza  não obedecem a relação causa e consequência, ou seja, apesar de não haver nenhuma mudança aparente em um determinado sistemas os resultados variam. Isso ocorre devido a aleatoriedade presente na natureza e uma forma de contornar isso é a repetição do experimento um número suficiente de vezes para se obter uma certa confiabilidade.
Por via de regra para o resultado ser 100% confiável o número de experimentos teria que tender ao infinito. Mas além de impraticável é também desnecessário na grande maioria dos casos. Nesse hands-on, cada repetição de **cenário** é chamado de **Job** e uma série de **jobs** formam uma **campanha** de simulação.


### **Como funciona?**
O SimPerScriptCampaign.ipynb é um arquivo que pode ser utilizado pelo jupyter-notebook ou Colab ([https://colab.research.google.com/?hl=pt_BR](https://colab.research.google.com/?hl=pt_BR)). Recomenda-se que seja usando em conjunto com o jupyter-notebook para que os arquivos das campanhas sejam criados localmente.

No arquivo SimPerScriptCampaign.ipynb, há um exemplo-teste, em que o usuário pode colocar diversas configurações para melhor entender o funcionamento dos scripts.

A criação da campanha envolve basicamente dois arquivos:

1. <SeuSistema>_SimPerScriptCampaign.py
2. <SeuSistema_SimPerScriptCampaign_<Variáveis1deControle>.yaml

No jupyter, os arquivos são chamados de **NrMmtc_SimPerScriptCampaign.py** e **NrMmtc_SimPerScriptCampaign_SimTeste.yaml,** porém, note que, podem ter qualquer nome. Tradicionalmente, usam-se nomes que ajudem na identificação tanto da campanha quanto do que está sendo simulado.

Sobre o arquivo .yaml, ele é um arquivo de texto com um formato específico, por exemplo:

- **NrMmtc_SimPerScriptCampaign_SimTeste.yaml**
 
    

Note que há uma hierarquia nesse arquivo de texto, isso é necessário para melhor distribuir e identificar os parâmetros do ns-3. Por exemplo, as hierarquias:

1. **ns3Parameters**: trata dos parâmetros relacionados ao diretório que o ns-3 está instalado, bem como o script (sem ".cc") que será utilizado na campanha. Alguns parâmetros não são utilizados, por não terem sido implementados ainda.
    
    a. **CampaignTag:** nome utilizado para identificar a campanha. É utilizado no momento em que se plota as curvas.
    
    b. **simLocation:** local onde serão realizadas as campanhas (**local,** **cluster, intel-256, intel-512, service**).
    
        i. Se **local**: o diretório do ns-3 deve ser colocado na variável: **ns3_path**
    
        ii. Se **cluster, intel-256, intel-512, service**: o diretório do ns-3 deve ser colocado na variável: **cluster_path**
    
2. **ShellScriptParameters:**
    
    a. **nOfCurlines**: informar a quantidade de campaignLines da simulação. Essa valor não pode ser maior do que a quantidade de variáveis informada.
    
    b. **SimTied**: se na simulação há alguma variável amarrada, isto é, que varia apenas se outra variar.
    
    c. **nOfCurlinesTied**: quantidade de variáveis amarrada. Não pode ser maior que nOfCurlines, se for menor considera-se apenas as primeiras amarradas iguais ao valor inserido.
    
        i. ex: nOfCurlines=3; SimTied=1; nOfCurlinesTied=2. Significa que apenas 2 das 3 variáveis estão amarradas, as 2 variáveis amarradas são as duas primeiras do campaignLines.
    
        ii. Configurações implementadas (nOfCurlines/SimTied/nOfCurlinesTied):
            1. 1/0/0
            2. 2/0/0
            3. 3/0/0
            4. 2/1/2
            5. 3/1/2
            6. 3/1/3
            7. 4/1/2
            8. 5/1/2
    
    d. **daytime**: quantidade de dias que a simulação levará no cluster
    
    e. **htime**: quantidade de horas que a simulação levará no cluster
    
    f. **ntasks**: quantidade de simulações  em 1 JOB 
    
        1. ATENÇÃO: cuidado para não errar esse valor.
    g. **cpusPerTask**: quantidade de CPUs necessárias para rodar a simulação no cluster. Cada CPU corresponde a 2 GB RAM
    
    h. **numberOfJobsShellScript**: quantidade de JOBs em um arquivo de submissão .sh. 
        1. ex: com base no exemplo mostrado, tem-se 20 JOBs. Então, serão criados 4 arquivos de submissão .sh, cada um contendo 5 JOBS.
    
3. **campaignLines:** trata dos parâmetros que variam na campanha. Informar qual (quais) parâmetro(s) comporá (ão) a(s) curva(s) do gráfico e qual (APENAS 1) ficará no eixo X dos gráficos.
    
4. **scenarioParameters:** parâmetros e seus valores, utilizados na sua simulação e campanha.
    a. ATENÇÃO: manter o mesmo padrão

### **Crie uma pasta para alocar os arquivos**

In [6]:
path = "/home/marilia/Campanha_Marilia" 
%cd $path

/home/marilia/Campanha_Marilia


### **Criando semente aleatória**
**Atenção**:Esse script foi criado para rodar apenas **uma** vez.Portanto se você ja possui o arquivo "ListOfRandomNumbers.pkl" **não** rode este script novamente.

### **Simulação da campanha (para o cmake)**

In [7]:
%%file SimPerScriptCampaign_Ns3Build.py
import os
import argparse
import pickle
import numpy as np

# To install YAML: sudo apt-get install python3-yaml
import yaml

class Simulation:
    def __init__(self, configurations_file):
        #self.email_to = 'sicrano@gmail.com'            
        with open(configurations_file, 'r') as f:
            self.doc = yaml.load(f, Loader=yaml.loader.BaseLoader)
        self.campaign_name = os.path.splitext(configurations_file)[0]
       
        # Simu parameters
        self.commandScript = ''
        for iscenarioParameters in self.doc['scenarioParameters'].items():
            commandraw= " --"+str(iscenarioParameters[0])+"="+str(iscenarioParameters[1][0])
            self.commandScript = self.commandScript + commandraw
        #print (self.commandScript)
    
        # ns-3 script configuration
        self.ns3_path = str(self.doc['ns3Parameters']['ns3_path'])
        self.ns3_path = os.getcwd() + '/' + self.ns3_path
        self.ns3_script = str(self.doc['ns3Parameters']['ns3_script'])
        self.CampaignTag = str(self.doc['ns3Parameters']['CampaignTag'])
        self.configurations_file = configurations_file       
        self.cluster_path = str(self.doc['ns3Parameters']['cluster_path'])
        self.simLocation = str(self.doc['ns3Parameters']['simLocation'])
        
        # Shell Script Parameters configuration
        self.nOfCurlines=self.doc['ShellScriptParameters']['nOfCurlines']
        self.SimTied = self.doc['ShellScriptParameters']['SimTied']
        self.nOfCurlinesTied = self.doc['ShellScriptParameters']['nOfCurlinesTied']
        self.daytime  = self.doc['ShellScriptParameters']['daytime']
        self.htime  = self.doc['ShellScriptParameters']['htime']
        self.ntasks = self.doc['ShellScriptParameters']['ntasks']
        self.cpusPerTask = self.doc['ShellScriptParameters']['cpusPerTask']
        self.numberOfJobsShellScript = int(self.doc['ShellScriptParameters']['numberOfJobsShellScript'])
        
         #Lines/curves
        self.campaignX = self.doc['campaignLines']['campaignX']
        self.campaignLines = self.doc['campaignLines']['campaignLines']
        self.nJobs = int(self.doc['campaignLines']['jobs'])
        
    def recursiveCommand(self,CurLine,scenarioParameters,vposition):
        command = (" --"+CurLine+"="+scenarioParameters[CurLine][vposition])
        return command
        
    
    def runCampaign(self,curCampaign):
        # Configure simulation file in accordance with campaign parameter
        numberOfJobsShellScript = self.numberOfJobsShellScript
        for iCallScripts in range (0,int(self.nJobs/numberOfJobsShellScript)):
            sh_name_main = self.campaign_name + '_' + self.simLocation + '_' + curCampaign + '_AllJOBS'+str(iCallScripts) +".sh"
            print("Main Shell Script: " + sh_name_main)  
            with open(sh_name_main, "w") as f:
                f.write("#!/bin/bash\n")
                #f.write("cd "+ self.cluster_path +"\n")
                for iJob in range(iCallScripts*numberOfJobsShellScript,iCallScripts*numberOfJobsShellScript + numberOfJobsShellScript):
                    for iSim in range(0, int(self.ntasks)):
                        sh_name = "run_"+self.campaign_name + '_' + self.simLocation + '_' + curCampaign + '_JOB' + str(iJob) + '_Sim_' + str(iSim)
                        if self.simLocation == 'cluster' or self.simLocation == 'service' or self.simLocation == 'intel-512' or self.simLocation == 'intel-256':
                            f.write("sbatch -p " + self.simLocation + " " + sh_name + ".sh" + "\n")
                        else:
                            f.write("chmod +x " + sh_name + ".sh" + " & wait\n")
                            f.write("./" + sh_name + ".sh" + " & wait\n") 
        for iJob in range(0,self.nJobs):         
            nOfCurlines=int(self.nOfCurlines[0]) # number of Campaign Lines in 1 simulation (max value = 3)
            SimTied = int(self.SimTied[0]) # whether or not the simulation parameters are tied (0 or 1)
            nOfCurlinesTied = int(self.nOfCurlinesTied[0]) # number of Campaign Lines tied (max value = nOfCurlines)
            with open('ListOfRandomNumbers.pkl', 'rb') as f:
                    jobRunSeed = pickle.load(f)
            count = 0
            """
            if SimTied == 0:
                minIterations=1
                for CampaignLines in self.doc['campaignLines']['campaignLines']:
                    CurlineIteration = len(self.doc['scenarioParameters'][CampaignLines])
                    totalCurlineIterations = minIterations*CurlineIteration
                    minIterations= totalCurlineIterations
                
                for varParam in self.doc['scenarioParameters'][curCampaign]:
                    tracker = [0]*nOfCurlines
                    for itotalCurlineIterations in range(totalCurlineIterations):
                        sh_name = self.campaign_name + '_' + self.simLocation + '_' + curCampaign + '_JOB' + str(iJob) + '_Sim_' + str(count)
                        print(curCampaign+" campaign written in file: " 'run_%s.sh' % sh_name)
                        with open('run_%s.sh' % sh_name, 'w') as f:                    
                            f.write('#!/bin/bash\n')
                            f.write('#SBATCH --ntasks=1\n')
                            f.write('#SBATCH --cpus-per-task='+self.cpusPerTask+'\n')
                            f.write('#SBATCH --time='+self.daytime+'-'+self.htime+':0 #especifica o tempo máximo de execução do job, dado no padrão dias-horas:minutos\n')
                            f.write('module load softwares/gsrc/2020_05\n')
                            outputDir = self.cluster_path+'results_'+self.simLocation + '_' + self.campaign_name + '_' + curCampaign
                            #f.write('rm -rf '+outputDir+"/JOB"+str(iJob)+' 2>/dev/null\n')
                            #for iJob in range(0, self.nJobs):
                            f.write('mkdir -p '+outputDir+"/JOB"+str(iJob)+"/Sim_"+str(count) +'\n')
                            f.write('cp -f run_'+sh_name+'.sh'+' '+outputDir+'\n')
                            f.write('cp -f '+self.configurations_file+ ' ' +outputDir+'\n')
                            f.write("cd '"+self.cluster_path+"'"+"\n")
                            command1 = (
                            'srun -N 1 -n 1' +
                            " ./waf --cwd='"+outputDir+"/JOB"+str(iJob)+"/Sim_"+str(count) +"'"
                            " --run '"+self.ns3_script+
                            " --RngRun="+str(jobRunSeed[iJob]))
                            command = ''
                            for iCampaignLines in range(len(self.doc['campaignLines']['campaignLines'])):
                                commandraw=self.recursiveCommand(self.doc['campaignLines']['campaignLines'][iCampaignLines], self.doc['scenarioParameters'],tracker[iCampaignLines])
                                command = command + commandraw
                                maxParamCurline = len(self.doc['scenarioParameters'][self.doc['campaignLines']['campaignLines'][iCampaignLines]])
                                print (command)
                                if iCampaignLines == 0 and tracker[iCampaignLines] < maxParamCurline:
                                    tracker[iCampaignLines] +=1
                                elif iCampaignLines == 0 and tracker[iCampaignLines] == maxParamCurline:
                                    tracker[iCampaignLines] = 0
                                
                                if iCampaignLines > 0:
                                    maxParamCurlinePrevious = len(self.doc['scenarioParameters'][self.doc['campaignLines']['campaignLines'][iCampaignLines-1]])
                                    if tracker[iCampaignLines-1] == maxParamCurlinePrevious:
                                        tracker[iCampaignLines] +=1
                                        tracker[iCampaignLines-1] =0
                            
                            command3 =(
                            " --outputDir='"+outputDir+"/JOB"+str(iJob)+"/Sim_"+str(count) +"'"+
                            " --outputDir2='"+outputDir+"/JOB"+str(iJob)+"'"+
                            command+
                            " --"+curCampaign+"="+varParam+
                            "' > "+outputDir+"/JOB"+str(iJob)+'/Sim_' + str(count)+'.out 2>&1')
                            f.write(command1+self.commandScript+command3+'\n')
                            count = count + 1
                          
            """
            if nOfCurlines == 1 and SimTied == 0:
                curline = self.campaignLines[0]
                count=0
                with open('ListOfRandomNumbers.pkl', 'rb') as f:
                    jobRunSeed = pickle.load(f)
                for i in range(len(self.doc['scenarioParameters'][curline])):    
                    for varParam in self.doc['scenarioParameters'][curCampaign]:
                        sh_name = self.campaign_name + '_' + self.simLocation + '_' + curCampaign + '_JOB' + str(iJob) + '_Sim_' + str(count)
                        print(curCampaign+" campaign written in file: " 'run_%s.sh' % sh_name)
                        with open('run_%s.sh' % sh_name, 'w') as f:                    
                            f.write('#!/bin/bash\n')
                            if self.simLocation == 'cluster' or self.simLocation == 'service' or self.simLocation == 'intel-512' or self.simLocation == 'intel-256':
                                outputDir = self.cluster_path+'results_'+self.simLocation + '_' + self.campaign_name + '_' + curCampaign
                                f.write('#SBATCH --time='+self.daytime+'-'+self.htime+':0 #especifica o tempo máximo de execução do job, dado no padrão dias-horas:minutos\n')
                                f.write('#SBATCH --ntasks=1\n')
                                f.write('#SBATCH --cpus-per-task='+self.cpusPerTask+'\n')
                                f.write('module load softwares/pacotao\n')
                                command1 = (
                                'srun -N 1 -n 1' +
                                " ./ns3 run '"+self.ns3_script+
                                " --RngRun="+str(jobRunSeed[iJob]))
                                f.write('mkdir -p '+outputDir+"/JOB"+str(iJob)+"/Sim_"+str(count) +'\n')
                                f.write('cp -f run_'+sh_name+'.sh'+' '+outputDir+'\n')
                                f.write('cp -f '+self.configurations_file+ ' ' +outputDir+'\n')
                                f.write("cd '"+self.cluster_path+"'"+"\n")
                                f.write("sleep $((11 + RANDOM % 50))"+"\n")
                            else:
                                outputDir = self.ns3_path+'results_'+self.simLocation + '_' + self.campaign_name + '_' + curCampaign
                                f.write('mkdir -p '+outputDir+"/JOB"+str(iJob)+"/Sim_"+str(count) +'\n')
                                f.write('cp -f run_'+sh_name+'.sh'+' '+outputDir+'\n')
                                f.write('cp -f '+self.configurations_file+ ' ' +outputDir+'\n')
                                f.write("cd '"+self.ns3_path+"'"+"\n")
                                f.write("sleep $((11 + RANDOM % 50))"+"\n")
                                command1 = (
                                " ./ns3 run '"+self.ns3_script+
                                " --RngRun="+str(jobRunSeed[iJob]))

                            #f.write('rm -rf '+outputDir+"/JOB"+str(iJob)+' 2>/dev/null\n')
                            #for iJob in range(0, self.nJobs):
                            command3 =(
                            " --outputDir='"+outputDir+"/JOB"+str(iJob)+"/Sim_"+str(count) +"'"+
                            " --outputDir2='"+outputDir+"/JOB"+str(iJob)+"'"+
                            " --"+curline+"="+self.doc['scenarioParameters'][curline][i]+
                            " --"+curCampaign+"="+varParam+"'"+
                            " --cwd='"+outputDir+"/JOB"+str(iJob)+"/Sim_"+str(count) +"'" +
                            " > "+outputDir+"/JOB"+str(iJob)+'/Sim_' + str(count)+'.out 2>&1')
                            f.write(command1+self.commandScript+command3+'\n')
                            count = count + 1
                                
            elif nOfCurlines==2 and SimTied == 0:
                with open('ListOfRandomNumbers.pkl', 'rb') as f:
                    jobRunSeed = pickle.load(f)
                curline = self.campaignLines[0]
                curline1 = self.campaignLines[1]
                count=0
                for i in range(len(self.doc['scenarioParameters'][curline])):
                    for j in range(len(self.doc['scenarioParameters'][curline1])):
                        for varParam in self.doc['scenarioParameters'][curCampaign]:
                            sh_name = self.campaign_name + '_' + self.simLocation + '_' + curCampaign + '_JOB' + str(iJob) + '_Sim_' + str(count)
                            print(curCampaign+" campaign written in file: " 'run_%s.sh' % sh_name)
                            with open('run_%s.sh' % sh_name, 'w') as f:                    
                                f.write('#!/bin/bash\n')
                                if self.simLocation == 'cluster' or self.simLocation == 'service' or self.simLocation == 'intel-512' or self.simLocation == 'intel-256':
                                    outputDir = self.cluster_path+'results_'+self.simLocation + '_' + self.campaign_name + '_' + curCampaign
                                    f.write('#SBATCH --time='+self.daytime+'-'+self.htime+':0 #especifica o tempo máximo de execução do job, dado no padrão dias-horas:minutos\n')
                                    f.write('#SBATCH --ntasks=1\n')
                                    f.write('#SBATCH --cpus-per-task='+self.cpusPerTask+'\n')
                                    f.write('module load softwares/pacotao\n')
                                    command1 = (
                                    'srun -N 1 -n 1' +
                                    " ./ns3 run '"+self.ns3_script+
                                    " --RngRun="+str(jobRunSeed[iJob]))
                                    f.write('mkdir -p '+outputDir+"/JOB"+str(iJob)+"/Sim_"+str(count) +'\n')
                                    f.write('cp -f run_'+sh_name+'.sh'+' '+outputDir+'\n')
                                    f.write('cp -f '+self.configurations_file+ ' ' +outputDir+'\n')
                                    f.write("cd '"+self.cluster_path+"'"+"\n")
                                    f.write("sleep $((11 + RANDOM % 50))"+"\n")
                                else:
                                    outputDir = self.ns3_path+'results_'+self.simLocation + '_' + self.campaign_name + '_' + curCampaign
                                    f.write('mkdir -p '+outputDir+"/JOB"+str(iJob)+"/Sim_"+str(count) +'\n')
                                    f.write('cp -f run_'+sh_name+'.sh'+' '+outputDir+'\n')
                                    f.write('cp -f '+self.configurations_file+ ' ' +outputDir+'\n')
                                    f.write("cd '"+self.ns3_path+"'"+"\n")
                                    f.write("sleep $((11 + RANDOM % 50))"+"\n")
                                    command1 = (
                                    " ./ns3 run '"+self.ns3_script+
                                    " --RngRun="+str(jobRunSeed[iJob]))
                                command3 =(
                                " --outputDir='"+outputDir+"/JOB"+str(iJob)+"/Sim_"+str(count) +"'"+
                                " --outputDir2='"+outputDir+"/JOB"+str(iJob)+"'"+
                                " --"+curline+"="+self.doc['scenarioParameters'][curline][i]+
                                " --"+curline1+"="+self.doc['scenarioParameters'][curline1][j]+
                                " --"+curCampaign+"="+varParam+"'"+
                                " --cwd='"+outputDir+"/JOB"+str(iJob)+"/Sim_"+str(count) +"'" +
                                " > "+outputDir+"/JOB"+str(iJob)+'/Sim_' + str(count)+'.out 2>&1')
                                f.write(command1+self.commandScript+command3+'\n')
                                count = count + 1
                            

            elif nOfCurlines == 3 and SimTied == 0:
                with open('ListOfRandomNumbers.pkl', 'rb') as f:
                    jobRunSeed = pickle.load(f)
                curline = self.campaignLines[0]
                curline1 = self.campaignLines[1]
                curline2 = self.campaignLines[2]
                count=0;
                for i in range(len(self.doc['scenarioParameters'][curline])):
                    for j in range(len(self.doc['scenarioParameters'][curline1])):
                        for k in range(len(self.doc['scenarioParameters'][curline2])):
                            for varParam in self.doc['scenarioParameters'][curCampaign]:
                                sh_name = self.campaign_name + '_' + self.simLocation + '_' + curCampaign + '_JOB' + str(iJob) + '_Sim_' + str(count)
                                print(curCampaign+" campaign written in file: " 'run_%s.sh' % sh_name)
                                with open('run_%s.sh' % sh_name, 'w') as f:                    
                                    f.write('#!/bin/bash\n')
                                    if self.simLocation == 'cluster' or self.simLocation == 'service' or self.simLocation == 'intel-512' or self.simLocation == 'intel-256':
                                        outputDir = self.cluster_path+'results_'+self.simLocation + '_' + self.campaign_name + '_' + curCampaign
                                        f.write('#SBATCH --time='+self.daytime+'-'+self.htime+':0 #especifica o tempo máximo de execução do job, dado no padrão dias-horas:minutos\n')
                                        f.write('#SBATCH --ntasks=1\n')
                                        f.write('#SBATCH --cpus-per-task='+self.cpusPerTask+'\n')
                                        f.write('module load softwares/pacotao\n')
                                        command1 = (
                                        'srun -N 1 -n 1' +
                                        " ./ns3 run '"+self.ns3_script+
                                        " --RngRun="+str(jobRunSeed[iJob]))
                                        f.write('mkdir -p '+outputDir+"/JOB"+str(iJob)+"/Sim_"+str(count) +'\n')
                                        f.write('cp -f run_'+sh_name+'.sh'+' '+outputDir+'\n')
                                        f.write('cp -f '+self.configurations_file+ ' ' +outputDir+'\n')
                                        f.write("cd '"+self.cluster_path+"'"+"\n")
                                        f.write("sleep $((11 + RANDOM % 50))"+"\n")
                                    else:
                                        outputDir = self.ns3_path+'results_'+self.simLocation + '_' + self.campaign_name + '_' + curCampaign
                                        f.write('mkdir -p '+outputDir+"/JOB"+str(iJob)+"/Sim_"+str(count) +'\n')
                                        f.write('cp -f run_'+sh_name+'.sh'+' '+outputDir+'\n')
                                        f.write('cp -f '+self.configurations_file+ ' ' +outputDir+'\n')
                                        f.write("cd '"+self.ns3_path+"'"+"\n")
                                        f.write("sleep $((11 + RANDOM % 50))"+"\n")
                                        command1 = (
                                        " ./ns3 run '"+self.ns3_script+
                                        " --RngRun="+str(jobRunSeed[iJob]))
                                    command3 =(
                                    " --outputDir='"+outputDir+"/JOB"+str(iJob)+"/Sim_"+str(count) +"'"+
                                    " --outputDir2='"+outputDir+"/JOB"+str(iJob)+"'"+
                                    " --"+curline+"="+self.doc['scenarioParameters'][curline][i]+
                                    " --"+curline1+"="+self.doc['scenarioParameters'][curline1][j]+
                                    " --"+curline2+"="+self.doc['scenarioParameters'][curline2][k]+
                                    " --"+curCampaign+"="+varParam+"'"+
                                    " --cwd='"+outputDir+"/JOB"+str(iJob)+"/Sim_"+str(count) +"'" +
                                    " > "+outputDir+"/JOB"+str(iJob)+'/Sim_' + str(count)+'.out 2>&1')
                                    f.write(command1+self.commandScript+command3+'\n')
                                    count = count + 1
            
            elif nOfCurlines == 2 and SimTied == 1 and nOfCurlinesTied == 2:
                with open('ListOfRandomNumbers.pkl', 'rb') as f:
                    jobRunSeed = pickle.load(f)
                curline = self.campaignLines[0]
                curline1 = self.campaignLines[1]
                #curline=self.doc['scenarioParameters'][campaignLines][0]
                #curline1=self.doc['scenarioParameters'][campaignLines][1]
                count=0
                for i in range(len(self.doc['scenarioParameters'][curline])):
                    for varParam in self.doc['scenarioParameters'][curCampaign]:
                        sh_name = self.campaign_name + '_' + self.simLocation + '_' + curCampaign + '_JOB' + str(iJob) + '_Sim_' + str(count)
                        print(curCampaign+" campaign written in file: " 'run_%s.sh' % sh_name)
                        with open('run_%s.sh' % sh_name, 'w') as f:
                            f.write('#!/bin/bash\n')
                            if self.simLocation == 'cluster' or self.simLocation == 'service' or self.simLocation == 'intel-512' or self.simLocation == 'intel-256':
                                outputDir = self.cluster_path+'results_'+self.simLocation + '_' + self.campaign_name + '_' + curCampaign
                                f.write('#SBATCH --time='+self.daytime+'-'+self.htime+':0 #especifica o tempo máximo de execução do job, dado no padrão dias-horas:minutos\n')
                                f.write('#SBATCH --ntasks=1\n')
                                f.write('#SBATCH --cpus-per-task='+self.cpusPerTask+'\n')
                                f.write('module load softwares/pacotao\n')
                                command1 = (
                                'srun -N 1 -n 1' +
                                " ./ns3 run '"+self.ns3_script+
                                " --RngRun="+str(jobRunSeed[iJob]))
                                f.write('mkdir -p '+outputDir+"/JOB"+str(iJob)+"/Sim_"+str(count) +'\n')
                                f.write('cp -f run_'+sh_name+'.sh'+' '+outputDir+'\n')
                                f.write('cp -f '+self.configurations_file+ ' ' +outputDir+'\n')
                                f.write("cd '"+self.cluster_path+"'"+"\n")
                                f.write("sleep $((11 + RANDOM % 50))"+"\n")
                            else:
                                outputDir = self.ns3_path+'results_'+self.simLocation + '_' + self.campaign_name + '_' + curCampaign
                                f.write('mkdir -p '+outputDir+"/JOB"+str(iJob)+"/Sim_"+str(count) +'\n')
                                f.write('cp -f run_'+sh_name+'.sh'+' '+outputDir+'\n')
                                f.write('cp -f '+self.configurations_file+ ' ' +outputDir+'\n')
                                f.write("cd '"+self.ns3_path+"'"+"\n")
                                f.write("sleep $((11 + RANDOM % 50))"+"\n")
                                command1 = (
                                " ./ns3 run '"+self.ns3_script+
                                " --RngRun="+str(jobRunSeed[iJob]))
                            command3 =(
                            " --outputDir='"+outputDir+"/JOB"+str(iJob)+"/Sim_"+str(count) +"'"+
                            " --outputDir2='"+outputDir+"/JOB"+str(iJob)+"'"+
                            " --"+curline+"="+self.doc['scenarioParameters'][curline][i]+
                            " --"+curline1+"="+self.doc['scenarioParameters'][curline1][i]+
                            " --"+curCampaign+"="+varParam+"'"+
                            " --cwd='"+outputDir+"/JOB"+str(iJob)+"/Sim_"+str(count) +"'" +
                            " > "+outputDir+"/JOB"+str(iJob)+'/Sim_' + str(count)+'.out 2>&1')
                            f.write(command1+self.commandScript+command3+'\n')
                            count = count + 1

                        
            elif nOfCurlines == 3 and SimTied == 1 and nOfCurlinesTied == 2:
                with open('ListOfRandomNumbers.pkl', 'rb') as f:
                    jobRunSeed = pickle.load(f)
                curline = self.campaignLines[0]
                curline1 = self.campaignLines[1]
                curline2 = self.campaignLines[2]
                count=0;
                for i in range(len(self.doc['scenarioParameters'][curline])):
                    for k in range(len(self.doc['scenarioParameters'][curline2])):
                        for varParam in self.doc['scenarioParameters'][curCampaign]:
                            sh_name = self.campaign_name + '_' + self.simLocation + '_' + curCampaign + '_JOB' + str(iJob) + '_Sim_' + str(count)
                            print(curCampaign+" campaign written in file: " 'run_%s.sh' % sh_name)
                            with open('run_%s.sh' % sh_name, 'w') as f:
                                f.write('#!/bin/bash\n')
                                if self.simLocation == 'cluster' or self.simLocation == 'service' or self.simLocation == 'intel-512' or self.simLocation == 'intel-256':
                                    outputDir = self.cluster_path+'results_'+self.simLocation + '_' + self.campaign_name + '_' + curCampaign
                                    f.write('#SBATCH --time='+self.daytime+'-'+self.htime+':0 #especifica o tempo máximo de execução do job, dado no padrão dias-horas:minutos\n')
                                    f.write('#SBATCH --ntasks=1\n')
                                    f.write('#SBATCH --cpus-per-task='+self.cpusPerTask+'\n')
                                    f.write('module load softwares/pacotao\n')
                                    command1 = (
                                    'srun -N 1 -n 1' +
                                    " ./ns3 run '"+self.ns3_script+
                                    " --RngRun="+str(jobRunSeed[iJob]))
                                    f.write('mkdir -p '+outputDir+"/JOB"+str(iJob)+"/Sim_"+str(count) +'\n')
                                    f.write('cp -f run_'+sh_name+'.sh'+' '+outputDir+'\n')
                                    f.write('cp -f '+self.configurations_file+ ' ' +outputDir+'\n')
                                    f.write("cd '"+self.cluster_path+"'"+"\n")
                                    f.write("sleep $((11 + RANDOM % 50))"+"\n")
                                else:
                                    outputDir = self.ns3_path+'results_'+self.simLocation + '_' + self.campaign_name + '_' + curCampaign
                                    f.write('mkdir -p '+outputDir+"/JOB"+str(iJob)+"/Sim_"+str(count) +'\n')
                                    f.write('cp -f run_'+sh_name+'.sh'+' '+outputDir+'\n')
                                    f.write('cp -f '+self.configurations_file+ ' ' +outputDir+'\n')
                                    f.write("cd '"+self.ns3_path+"'"+"\n")
                                    f.write("sleep $((11 + RANDOM % 50))"+"\n")
                                    command1 = (
                                    " ./ns3 run '"+self.ns3_script+
                                    " --RngRun="+str(jobRunSeed[iJob]))
                                command3 =(
                                " --outputDir='"+outputDir+"/JOB"+str(iJob)+"/Sim_"+str(count) +"'"+
                                " --outputDir2='"+outputDir+"/JOB"+str(iJob)+"'"+
                                " --"+curline+"="+self.doc['scenarioParameters'][curline][i]+
                                " --"+curline1+"="+self.doc['scenarioParameters'][curline1][i]+
                                " --"+curline2+"="+self.doc['scenarioParameters'][curline2][k]+
                                " --"+curCampaign+"="+varParam+"'"+
                                " --cwd='"+outputDir+"/JOB"+str(iJob)+"/Sim_"+str(count) +"'" +
                                " > "+outputDir+"/JOB"+str(iJob)+'/Sim_' + str(count)+'.out 2>&1')
                                f.write(command1+self.commandScript+command3+'\n')
                                count = count + 1
            
            elif nOfCurlines == 3 and SimTied == 1 and nOfCurlinesTied == 3:
                with open('ListOfRandomNumbers.pkl', 'rb') as f:
                    jobRunSeed = pickle.load(f)
                curline0 = self.campaignLines[0]
                curline1 = self.campaignLines[1]
                curline2 = self.campaignLines[2]
                count=0
                for i in range(len(self.doc['scenarioParameters'][curline])):
                    for varParam in self.doc['scenarioParameters'][curCampaign]:
                        sh_name = self.campaign_name + '_' + self.simLocation + '_' + curCampaign + '_JOB' + str(iJob) + '_Sim_' + str(count)
                        print(curCampaign+" campaign written in file: " 'run_%s.sh' % sh_name)
                        with open('run_%s.sh' % sh_name, 'w') as f:
                            f.write('#!/bin/bash\n')
                            if self.simLocation == 'cluster' or self.simLocation == 'service' or self.simLocation == 'intel-512' or self.simLocation == 'intel-256':
                                outputDir = self.cluster_path+'results_'+self.simLocation + '_' + self.campaign_name + '_' + curCampaign
                                f.write('#SBATCH --time='+self.daytime+'-'+self.htime+':0 #especifica o tempo máximo de execução do job, dado no padrão dias-horas:minutos\n')
                                f.write('#SBATCH --ntasks=1\n')
                                f.write('#SBATCH --cpus-per-task='+self.cpusPerTask+'\n')
                                f.write('module load softwares/pacotao\n')
                                command1 = (
                                'srun -N 1 -n 1' +
                                " ./ns3 run '"+self.ns3_script+
                                " --RngRun="+str(jobRunSeed[iJob]))
                                f.write('mkdir -p '+outputDir+"/JOB"+str(iJob)+"/Sim_"+str(count) +'\n')
                                f.write('cp -f run_'+sh_name+'.sh'+' '+outputDir+'\n')
                                f.write('cp -f '+self.configurations_file+ ' ' +outputDir+'\n')
                                f.write("cd '"+self.cluster_path+"'"+"\n")
                                f.write("sleep $((11 + RANDOM % 50))"+"\n")
                            else:
                                outputDir = self.ns3_path+'results_'+self.simLocation + '_' + self.campaign_name + '_' + curCampaign
                                f.write('mkdir -p '+outputDir+"/JOB"+str(iJob)+"/Sim_"+str(count) +'\n')
                                f.write('cp -f run_'+sh_name+'.sh'+' '+outputDir+'\n')
                                f.write('cp -f '+self.configurations_file+ ' ' +outputDir+'\n')
                                f.write("cd '"+self.ns3_path+"'"+"\n")
                                f.write("sleep $((11 + RANDOM % 50))"+"\n")
                                command1 = (
                                " ./ns3 run '"+self.ns3_script+
                                " --RngRun="+str(jobRunSeed[iJob]))
                            command3 =(
                            " --outputDir='"+outputDir+"/JOB"+str(iJob)+"/Sim_"+str(count) +"'"+
                            " --outputDir2='"+outputDir+"/JOB"+str(iJob)+"'"+
                            " --"+curline+"="+self.doc['scenarioParameters'][curline][i]+
                            " --"+curline1+"="+self.doc['scenarioParameters'][curline1][i]+
                            " --"+curline2+"="+self.doc['scenarioParameters'][curline2][i]+
                            " --"+curCampaign+"="+varParam+"'"+
                            " --cwd='"+outputDir+"/JOB"+str(iJob)+"/Sim_"+str(count) +"'" +
                            " > "+outputDir+"/JOB"+str(iJob)+'/Sim_' + str(count)+'.out 2>&1')
                            f.write(command1+self.commandScript+command3+'\n')
                            count = count + 1
            
            elif nOfCurlines == 4 and SimTied == 1 and nOfCurlinesTied == 2:
                with open('ListOfRandomNumbers.pkl', 'rb') as f:
                    jobRunSeed = pickle.load(f)
                curline = self.campaignLines[0]
                curline1 = self.campaignLines[1]
                curline2 = self.campaignLines[2]
                curline3 = self.campaignLines[3]
                count=0
                for i in range(len(self.doc['scenarioParameters'][curline])):
                    for k in range(len(self.doc['scenarioParameters'][curline2])):
                        for n in range(len(self.doc['scenarioParameters'][curline3])):
                            for varParam in self.doc['scenarioParameters'][curCampaign]:
                                sh_name = self.campaign_name + '_' + self.simLocation + '_' + curCampaign + '_JOB' + str(iJob) + '_Sim_' + str(count)
                                print(curCampaign+" campaign written in file: " 'run_%s.sh' % sh_name)
                                with open('run_%s.sh' % sh_name, 'w') as f:
                                    f.write('#!/bin/bash\n')
                                    if self.simLocation == 'cluster' or self.simLocation == 'service' or self.simLocation == 'intel-512' or self.simLocation == 'intel-256':
                                        outputDir = self.cluster_path+'results_'+self.simLocation + '_' + self.campaign_name + '_' + curCampaign
                                        f.write('#SBATCH --time='+self.daytime+'-'+self.htime+':0 #especifica o tempo máximo de execução do job, dado no padrão dias-horas:minutos\n')
                                        f.write('#SBATCH --ntasks=1\n')
                                        f.write('#SBATCH --cpus-per-task='+self.cpusPerTask+'\n')
                                        f.write('module load softwares/pacotao\n')
                                        command1 = (
                                        'srun -N 1 -n 1' +
                                        " ./ns3 run '"+self.ns3_script+
                                        " --RngRun="+str(jobRunSeed[iJob]))
                                        f.write('mkdir -p '+outputDir+"/JOB"+str(iJob)+"/Sim_"+str(count) +'\n')
                                        f.write('cp -f run_'+sh_name+'.sh'+' '+outputDir+'\n')
                                        f.write('cp -f '+self.configurations_file+ ' ' +outputDir+'\n')
                                        f.write("cd '"+self.cluster_path+"'"+"\n")
                                        f.write("sleep $((11 + RANDOM % 50))"+"\n")
                                    else:
                                        outputDir = self.ns3_path+'results_'+self.simLocation + '_' + self.campaign_name + '_' + curCampaign
                                        f.write('mkdir -p '+outputDir+"/JOB"+str(iJob)+"/Sim_"+str(count) +'\n')
                                        f.write('cp -f run_'+sh_name+'.sh'+' '+outputDir+'\n')
                                        f.write('cp -f '+self.configurations_file+ ' ' +outputDir+'\n')
                                        f.write("cd '"+self.ns3_path+"'"+"\n")
                                        f.write("sleep $((11 + RANDOM % 50))"+"\n")
                                        command1 = (
                                        " ./ns3 run '"+self.ns3_script+
                                        " --RngRun="+str(jobRunSeed[iJob]))
                                    command3 =(
                                    " --outputDir='"+outputDir+"/JOB"+str(iJob)+"/Sim_"+str(count) +"'"+
                                    " --outputDir2='"+outputDir+"/JOB"+str(iJob)+"'"+
                                    " --"+curline+"="+self.doc['scenarioParameters'][curline][i]+
                                    " --"+curline1+"="+self.doc['scenarioParameters'][curline1][i]+
                                    " --"+curline2+"="+self.doc['scenarioParameters'][curline2][k]+
                                    " --"+curline3+"="+self.doc['scenarioParameters'][curline3][n]+
                                    " --"+curCampaign+"="+varParam+"'"+
                                    " --cwd='"+outputDir+"/JOB"+str(iJob)+"/Sim_"+str(count) +"'" +
                                    " > "+outputDir+"/JOB"+str(iJob)+'/Sim_' + str(count)+'.out 2>&1')
                                    f.write(command1+self.commandScript+command3+'\n')
                                    count = count + 1
            
            elif nOfCurlines == 5 and SimTied == 1 and nOfCurlinesTied == 2:
                with open('ListOfRandomNumbers.pkl', 'rb') as f:
                    jobRunSeed = pickle.load(f)
                curline = self.campaignLines[0]
                curline1 = self.campaignLines[1]
                curline2 = self.campaignLines[2]
                curline3 = self.campaignLines[3]
                curline4 = self.campaignLines[4]
                count=0
                for i in range(len(self.doc['scenarioParameters'][curline])):
                    for k in range(len(self.doc['scenarioParameters'][curline2])):
                        for n in range(len(self.doc['scenarioParameters'][curline3])):
                            for h in range(len(self.doc['scenarioParameters'][curline4])):
                                for varParam in self.doc['scenarioParameters'][curCampaign]:
                                    sh_name = self.campaign_name + '_' + self.simLocation + '_' + curCampaign + '_JOB' + str(iJob) + '_Sim_' + str(count)
                                    print(curCampaign+" campaign written in file: " 'run_%s.sh' % sh_name)
                                    with open('run_%s.sh' % sh_name, 'w') as f:
                                        f.write('#!/bin/bash\n')
                                        if self.simLocation == 'cluster' or self.simLocation == 'service' or self.simLocation == 'intel-512' or self.simLocation == 'intel-256':
                                            outputDir = self.cluster_path+'results_'+self.simLocation + '_' + self.campaign_name + '_' + curCampaign
                                            f.write('#SBATCH --time='+self.daytime+'-'+self.htime+':0 #especifica o tempo máximo de execução do job, dado no padrão dias-horas:minutos\n')
                                            f.write('#SBATCH --ntasks=1\n')
                                            f.write('#SBATCH --cpus-per-task='+self.cpusPerTask+'\n')
                                            f.write('module load softwares/pacotao\n')
                                            command1 = (
                                            'srun -N 1 -n 1' +
                                            " ./ns3 run '"+self.ns3_script+
                                            " --RngRun="+str(jobRunSeed[iJob]))
                                            f.write('mkdir -p '+outputDir+"/JOB"+str(iJob)+"/Sim_"+str(count) +'\n')
                                            f.write('cp -f run_'+sh_name+'.sh'+' '+outputDir+'\n')
                                            f.write('cp -f '+self.configurations_file+ ' ' +outputDir+'\n')
                                            f.write("cd '"+self.cluster_path+"'"+"\n")
                                            f.write("sleep $((11 + RANDOM % 50))"+"\n")
                                        else:
                                            outputDir = self.ns3_path+'results_'+self.simLocation + '_' + self.campaign_name + '_' + curCampaign
                                            f.write('mkdir -p '+outputDir+"/JOB"+str(iJob)+"/Sim_"+str(count) +'\n')
                                            f.write('cp -f run_'+sh_name+'.sh'+' '+outputDir+'\n')
                                            f.write('cp -f '+self.configurations_file+ ' ' +outputDir+'\n')
                                            f.write("cd '"+self.ns3_path+"'"+"\n")
                                            f.write("sleep $((11 + RANDOM % 50))"+"\n")
                                            command1 = (
                                            " ./ns3 run '"+self.ns3_script+
                                            " --RngRun="+str(jobRunSeed[iJob]))
                                        command3 =(
                                        " --outputDir='"+outputDir+"/JOB"+str(iJob)+"/Sim_"+str(count) +"'"+
                                        " --outputDir2='"+outputDir+"/JOB"+str(iJob)+"'"+
                                        " --"+curline+"="+self.doc['scenarioParameters'][curline][i]+
                                        " --"+curline1+"="+self.doc['scenarioParameters'][curline1][i]+
                                        " --"+curline2+"="+self.doc['scenarioParameters'][curline2][k]+
                                        " --"+curline3+"="+self.doc['scenarioParameters'][curline3][n]+
                                        " --"+curline4+"="+self.doc['scenarioParameters'][curline4][h]+
                                        " --"+curCampaign+"="+varParam+"'"+
                                        " --cwd='"+outputDir+"/JOB"+str(iJob)+"/Sim_"+str(count) +"'" +
                                        " > "+outputDir+"/JOB"+str(iJob)+'/Sim_' + str(count)+'.out 2>&1')
                                        f.write(command1+self.commandScript+command3+'\n')
                                        count = count + 1

            #f.write('wait')
                                                

parser = argparse.ArgumentParser()
parser.add_argument("-f", "--file", type=str, help='Configuration File')
args = parser.parse_args()

configurations_file = args.file; 
with open(configurations_file, 'r') as f:
    doc = yaml.load(f, Loader=yaml.loader.BaseLoader)
    campaign_name = os.path.splitext(configurations_file)[0]
"""
doc = {'ns3Parameters': {'ns3_path': '',
  'ns3_script': 'NrMmtc-M0_v10_REM',
  'cluster_path': '/home/drdluna/5G-LENA/ns-3-dev-nr-1.2-rem/',
  'filename': 'CampaignNrCa',
  'simLocation': 'cluster'},
 'ShellScriptParameters': {'nOfCurlines': '3',
  'SimTied': '0',
  'nOfCurlinesTied': '0',
  'daytime': '1',
  'htime': '6',
  'ntasks': '8',
  'cpusPerTask': '1',
  'numberOfJobsShellScript': '1'},
 'campaignLines': {'campaignX': ['totalUePower'],
  'campaignLines': ['totalTxPower', 'DowntiltAngleDegree','BearingAngleTilt'],
  'jobs': '1'},
 'scenarioParameters': {'window': ['0.1'],
  'simTime': ['15'],
  'Isd': ['500'],
  'ueNumScenario': ['84'],
  'centralFrequencyBand': ['2350000000'],
  'bandwidthBand': ['100000000'],
  'BwpConfiguration': ['1'],
  'BwpBand': ['72000000'],
  'remBwpId': ['0'],
  'scenario': ['UmaLos'],
  'Shadowing': ['0'],
  'ChannelUpdate': ['0'],
  'totalTxPower': ['32', '26'],
  'totalUePower': ['23', '17'],
  'DowntiltAngleDegree': ['0', '15'],
  'BearingAngleTilt': ['0','-30'],
  'UlMode': ['1'],
  'TrafficModel': ['TriggeredReport'],
  'numerology': ['0'],
  'PacketSize': ['125'],
  'interval': ['60'],
  'Lambda': ['0.1']}}

configurations_file = "SimPerScriptCampaign_Teste.yaml"
"""
print('Simulação escolhida: ')
campaign = doc['campaignLines']['campaignX']
print(campaign)
                 
simu = Simulation(configurations_file)

for simC in campaign:
    simu.runCampaign(simC)
    


Overwriting SimPerScriptCampaign_Ns3Build.py


In [8]:
%cd $path
!mkdir "SimTest"
%cp -f "SimPerScriptCampaign_Ns3Build.py" $path"/SimTest"
%cp -f "ListOfRandomNumbers.pkl" $path"/SimTest"
%cd "SimTest"

/home/marilia/Campanha_Marilia
/home/marilia/Campanha_Marilia/SimTest


### **NS3 parâmetros**

In [9]:
%%file NrMmtc_SimPerScriptCampaign_SimTest.yaml
# ns-3 script configuration
ns3Parameters:

    ns3_path:  /home/marilia/ns-allinone-3.36/ns-3.36 # Complete path of ns-3 simulator
    ns3_script: my-lena-simple-epc   # Script with the main code (inside scratch folder without .cc)
    cluster_path: /home/mcmuniz/ns-allinone-3.36/ns-3.36/ #MUDAR
    CampaignTag: CampaignLte_EPC          # Output filename
    simLocation: intel-256            # Set location of simulation (local or cluster)

# Shell Script Parameters configuration
ShellScriptParameters: 
    
    nOfCurlines: 3      # number of Campaign Lines
    SimTied: 0
    nOfCurlinesTied: 0  # if greater than nOfCurlines, means that campaignX is included
    daytime: 1          # number of days to simulate (only for cluster simulation) 
    htime: 12            # number of hours to simulate (only for cluster simulation) 
    ntasks: 36           # number of simulation in one .sh (only for cluster simulation - default: 30) 
    cpusPerTask: 1      # numer of CPUs per Task
    numberOfJobsShellScript: 2

#Lines/curves
campaignLines:

    campaignX:          # campaignX: parameter name to vary in the X-axis (ONLY 1)
        - lambda
    campaignLines:      # campaignLines: parameter name to vary in different lines
        - numUes
        - useShadowing
        - useFading
    jobs: 10             # Number of jobs to run 
        
# Simu parameters
# colocar apenas os parâmetros que podem ser alterados via linha de comando
scenarioParameters:
    lambda:
        - 0.001
        - 0.01
        - 0.1
    packetSize:
        - 125
    numEnbs:
        - 1
    numUes:
        - 1
        - 5
        - 10
    simTime:
        - 10.0
    distance:
        - 20
    interPacketInterval:
        - 100
    radius:
        - 10.0
    useUdp:
        - 0
    usePoisson:
        - 1
    useShadowing:
        - 1
        - 0
    useFading:
        - 1
        - 0
        
    useDl:
        - 1
    useUl:
        - 0
    usePl:
        - 0
    verbose:
        - 0



Writing NrMmtc_SimPerScriptCampaign_SimTest.yaml


# **Rodando uma campanha no super computador**

Tendo gerado todos os arquivos corretamente, o próximo passo é copiar a pasta "Campanha" para dentro do ns3 previamente instalado no super pc.Lembrando que após adicionar a pasta "Campanha" dentro da pasta raiz do ns3 é necessário buildar novamente.
   
Para rodar os arquivos allJOB no super pc é necessário entrar no diretorio /home/mcmuniz/ns-allinone-3.36/ns-3.36/Campanhas/SimTest e usar os comandos abaixo:
   
        chmod +x NrMmtc_SimPerScriptCampaign_SimTest_intel-256_lambda_AllJOBS1.sh
        
        ./NrMmtc_SimPerScriptCampaign_SimTest_intel-256_lambda_AllJOBS1.sh
    
Ao final será criada,dentro do /home/mcmuniz/ns-allinone-3.36/ns-3.36,uma pasta chamada"results_intel256_NrMmtc_SimPerScriptCampaign_SimTest_lambda" contendo os resultados da campanha.
![29.png](FIGS/29.png)



![28.png](FIGS/28.png)




# **Resultados da campanha utilizando LENA**

### **Objetivo:**
A campanha de simulação visa obter resultados sobre o comportamento do LTE para aumento do número de usuários e do tráfego na comunicação em cenários com atenuação de caminho (Pathloss sempre ativo).

### **Cenário:**

O cenário é composto por um enlace eNB - UE onde a distância máxima que os UEs são posicionados é a 100 m da posição da eNB e são consideradas as influências de efeitos de *Fading* e *Shadowing*.

### **Metodologia:**

Desta forma, as campanhas possuem as seguintes linhas:
* Número de UEs:
    * 1
    * 5
    * 10
* Uso do Shadowing:
    * Ativado
    * Desativado
* Uso do Fading:
    * Ativado
    * Desativado
    
Além disso, possuem 3 diferentes lambdas, variável que controla o tráfego do enlace, sendo eles:
* Tamanho do lambda: 
    * 0.001
    * 0.01
    * 0.1

Assim, 10 JOBs foram realizados, cada um possuindo 36 simulações, devido ao número de variáveis que foi constituído. Após terem sido plotados os gráficos de convergência de métricas, constatou-se que, para esse cenário, 10 JOBs foram suficientes para os dados convergirem.

## **Gráficos:**

Para ter acesso aos scripts de plot basta clicar nesse [link](https://github.com/MariliaCostaMuniz/ic_LTE/tree/main/fase_01/HD_04#:~:text=last%20week-,PlotGraphs_v3.ipynb,-(File)).Nele esta contido todo o passo a passo para gerar os gráficos.

### ***HARQ* (Solicitação de Repetição Automática Híbrida)**

Enquanto a *ARQ* refere-se à solicitação de repetição automática, ou seja, se o remetente não receber a confirmação (*ACK*) antes do tempo limite, o receptor descarta o pacote ruim e o remetente deverá retransmitir o pacote. O processo *HARQ* depende do recebimento de *ACK* para os pacotes. 

Se o remetente envia um pacote e depois espera o *ACK* enviar outro pacote, ele é chamado de processo *SAW* - *Stop and Wait* (Pare E Espere). O tempo de ida e volta será maior (tempo de processamento do emissor e do receptor + atrasos de propagação). Assim, vários processos *SAW* são usados em **LTE**, por exemplo, quando 1 processo SAW está aguardando *ACK*, outro processo SAW pode enviar os dados. Esses processos *SAW* também são conhecidos como processos *HARQ*.

#### **Número de retransmissões *HARQ* - Lambda**

Selecionando apenas as curvas onde useFading é igual a 0, enxerga-se que o número de retransmissões HARQ é igual a zero, independentemente da presença de shadowing. 

Já ao selecionar as curvas com useShadowing igual a 0 e useFading igual a 1 vê-se que o valor de *HARQ* é diferente de zero e decai à medida que o lambda aumenta. Além disso, infere-se que sistemas com mais UEs possuem maior número de retransmissões, o que vai ao encontro com o esperado de um sistema com a presença de Fading no canal e que passa a ter mais usuários.

Quando compara-se as curvas com useShadowing igual a 0 com as com useShadowing igual a 1, quando ambas possuem useFading igual a 1, percebe-se que nessa situação o *Shadowing* vai estar agindo negativamente pro sistema uma vez que, para um mesmo valor de numUes, o número de retransmissões aumenta.

In [1]:
from IPython.display import IFrame
IFrame(src='html/Harq-System_Lambda.html', width=1000, height=500)

#### ***HARQ CDF - HARQ***

Os gráficos de CDF demonstram os valores utilizados para o anterior que utiliza a média, dessa forma, não existe nenhuma diferença notável aqui pois os valores de retransmissões *HARQ* obtidos são muito pequenos.

In [2]:
from IPython.display import IFrame
IFrame(src='html/Harq-System-CDF_AllCurves_Lambda.html', width=1000, height=500)

### ***MCS* (Modulação e Esquema de Codificação)**

O MCS define quanto de taxa útil pode ser transmitido por *Resource Element* (RE), como esse parâmetro depende da qualidade do enlace de rádio, quanto melhor a qualidade, maior o MCS e mais taxa útil terá a transmissão.

#### ***System MCS* - Lambda**

Desta forma observa-se que, independentemente do valor de numUes, quando useShadowing e useFading são iguais a zero, o MCS permanece constante em um valor de aproximadamente 28 que é o máximo valor para uma modulação 64QAM (melhor cenário possível).

Todavia, em todas as curvas em que useFading é igual a 1 tem-se valores de MCS inferiores a 28 para ambos os 3 casos de Lambda.

Por outro lado, ao visualizar as curvas com useShadowing igual a 1 e use Fading igual a 0, e posteriormente compará-las com as que possuem tanto useShadowing como useFading iguais a 1,  vê-se que o *Shadowing* não irá causar queda do valor de MCS por si só, mas irá acentuar a queda do valor do MCS quando presente juntamente com o *Fading*.

In [3]:
from IPython.display import IFrame
IFrame(src='html/MCS-System_Lambda.html', width=1000, height=500)

#### ***System MCS CDF - MCS***

In [4]:
from IPython.display import IFrame
IFrame(src='html/MCS-System-CDF_AllCurves_Lambda.html', width=1000, height=500)

### ***PLR* (Porcentagem de Perda de Pacote)**

Representa a relação de pacotes perdidos por pacotes enviados em porcentagem.

#### ***IP PLR* - Lambda**

Além dos valores **muito** pequenos de *PLR*, é notável que as curvas estão sobrepostas, dando a impressão que existem apenas 3 curvas. A partir dessa visualização, constata-se que a quantidade de UEs é a única variável que afeta o *PLR*. Para os 3 conjuntos de curvas a simulação apresentou resultados que decaem à medida que o valor do lambda é aumentado. 

É importante ressaltar que mesmo havendo variação nos valores eles ainda são praticamente nulos para ambos os 3 valores de número de UEs, chegando a ser realmente zero quando o lambda é igual a 0.1.

O resultado está dentro do esperado uma vez que os cenários construídos não são degradados o suficiente para trazer prejuízos à comunicação ao ponto de se ter perda de pacotes relevantes.

In [5]:
from IPython.display import IFrame
IFrame(src='html/PLR-IP_Lambda.html', width=1000, height=500)

#### ***IP PLR CDF - Lambda***

Conforme esperava-se, baseado no gráfico anterior, os valores em porcentagem da CDF são extremamente próximos 100% a partir de 0. Porém, ao dar zoom, percebemos que os gráficos não são tão certos assim, isso se deve pois os valores de PLR, apesar de muito próximos, não são exatamente iguais a zero.

In [6]:
from IPython.display import IFrame
IFrame(src='html/PLR-IP-CDF_AllCurves_Lambda.html', width=1000, height=500)

### ***SINR* (Relação Sinal Ruído Interferência)**

Diz respeito à relação entre a potência do sinal sobre a potência do ruído + interferência, dada em dB.

#### ***System SINR CDF - SINR***

Há diversas maneiras de se analisar este cenário sobre a perspectiva da relação sinal-ruído interferência. Inicialmente, pode-se visualizar que o valor de lambda não irá influenciar nas curvas de SINR, uma vez que, para o mesmo valor de numUes, useFading e useShadowing, as curvas têm o mesmo comportamento independente do valor de lambda. 

Adotando como base o patamar de 60% da CDF, tem-se que o cenário com melhor SINR é o que possui numUes igual a 1, useShadowing igual a 1 e useFading igual a 1. Em contrapartida, os piores cenários para esse patamar são os com numUes igual a 1, useShadowing igual a 0, independente do valor de useFading e de Lambda (ambos possuem o mesmo valor de SINR em 60% da CDF).

Agora, comparando as curvas de useShadowing e useFading iguais a 0 com as que possuem useShadowing igual a 1, para qualquer valor de Lambda e numUes vê-se que as curvas que possuem useShadowing igual a 1 entregam um maior valor de SINR na maioria dos casos.

A partir dessas duas análises levanta-se a hipótese de que o Shadowing presente no canal está se somando ao sinal transmitido, ao invés de diminuir sua potência, o que de fato pode acontecer uma vez que se trata de uma variável aleatória.

Analisando o caso de numUes igual a 10 e Lambda igual da 0.001, a diferença entre o pior caso (apenas useFading igual a 1) e o pior caso (useFading e useShadowing iguais a 1), quando a CDF está em 20%, por exemplo, é de apenas 1.13 dB. Isso nos leva a crer que, apesar de terem sido inseridos efeitos de perda de propagação, o cenário ainda está longe de chegar em um patamar onde a comunicação será de fato comprometida por estes.

In [7]:
from IPython.display import IFrame
IFrame(src='html/SINR-System-CDF_AllCurves_Lambda.html', width=1000, height=500)

### ***Tput (Throughput)***

O *Throughput* é a taxa de transferência de rede em kbps.

#### ***IP Tput*- Lambda**

É simples perceber que Lambda é a única variável que irá afetar o valor do Tput de forma notável. Como esperado do comportamento do sistema, à medida que o Lambda aumenta, o valor de Tput cai, pois o tempo de espera entre pacotes é maior.

Na prática existe diferença do valor de Tput (visualiza-se isso dando zoom nos pontos), considerando os diferentes valores de numUes, entretanto, a diferença é muito pequena.

In [8]:
from IPython.display import IFrame
IFrame(src='html/Tput-IP_Lambda.html', width=1000, height=500)

#### ***IP Tput CDF - Tput***

Em CDF é possível observar a mesma situação onde os valores de cada Lambda estão posicionados sobre seus respectivos valores de Tput.

In [9]:
from IPython.display import IFrame
IFrame(src='html/Tput-IP-CDF_AllCurves_Lambda.html', width=1000, height=500)